In [1]:
import os
from tqdm.auto import tqdm, trange
import argparse
import random
import pickle
import numpy as np
import pandas as pd
import json
from pprint import pprint
from glob import glob
from datasets import load_dataset, Dataset

/home/jhw/.venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
files = glob('../data/*/*/*.json')

In [4]:
def read_json(file):
    with open(file, 'rb') as f:
        data = json.load(f)
    return data

def extract_train_data(data):
    relation = data['info']['relation']
    situation = data['info']['situation']
    conversation = [
        {
            'role' : 'user' if x['role'] == 'speaker' else 'assistant',
            'content' : x['text'].replace('감정화자','너')
        } for x in data['utterances']
    ]

    return relation, situation, conversation

def make_trainset(files):
    relations = []
    situations = []
    conversations = []
    for file in tqdm(files):
        data = read_json(file)
        if data['info']['evaluation']['avg_rating'] >= 5:
            relation, situation, conversation = extract_train_data(data)
            if relation in ['친구']:
                relations.append(relation)
                situations.append(situation)
                conversations.append(conversation)
    
    output = {
        'relation' : relations,
        'situation' : situations,
        'conversation' : conversations
    }
    return output

In [5]:
trainset = make_trainset(files)

100%|██████████| 28638/28638 [00:02<00:00, 11847.19it/s]


In [6]:
i = 4
trainset['situation'][i], trainset['conversation'][i]

('취미로 베이킹을 해서 주변에 나눠준다.',
 [{'role': 'user', 'content': '혹시 오늘 바빠? 안 바쁘면 퇴근하고 나랑 잠깐 볼래?'},
  {'role': 'assistant', 'content': '오늘 안 바빠서 완전 칼퇴야. 근데 무슨 일 있어?'},
  {'role': 'user',
   'content': '아니 다른 게 아니고 내가 취미로 베이킹하잖아. 오늘 아침에 빵을 좀 구워서 주변에 나눠주려고 들고 왔거든. 너도 좀 나눠주려고.'},
  {'role': 'assistant',
   'content': '정말? 나 완전 빵순이잖아. 정말 기대된다. 너무 좋아. 벌써 퇴근하고 싶어서 미치겠어.'},
  {'role': 'user',
   'content': '주변에서 이렇게 좋아할 때마다 너무 뿌듯해서 계속 만들고 나눠주게 되는 것 같아. 내 마음이 너무 풍족해지는 기분이라 너무 기뻐.'},
  {'role': 'assistant',
   'content': '네가 기쁘다니 나도 기분이 너무 좋다. 그나저나 나눔을 통해서 기쁨을 얻는다니 네가 너무 대단하게 느껴지는걸?'},
  {'role': 'user',
   'content': '나눔을 통해서 얻는 기쁨은 값을 매길 수 없는 것 같아. 난 취미를 하면서 남도 행복하게 만들어 줄 수 있어서 너무 행복해.'},
  {'role': 'assistant',
   'content': '자신에게 맞는 취미를 찾는 게 쉬운 일이 아닌데 너무 잘된 일이다. 너를 그렇게 행복하게 만드는 취미를 찾아서 나까지 너무 기쁘다.'},
  {'role': 'user',
   'content': '그렇게 말해줘서 고마워. 나눠주다 보면 이렇게 돈 쓰고 시간 써가면서 왜 다 남한테 주냐고 그런 사람들도 있거든. 네가 이해해 주니까 더 행복하다 헤헤.'},
  {'role': 'assistant',
   'content': '나누다 보면 받은 사람들도 받은 만큼 보답하고 싶기 마련이잖아. 굳이 너

In [15]:
len('\n'.join([x['text'] for x in trainset['conversation'][0]]))

1776

In [7]:
dataset = Dataset.from_dict(trainset)

In [9]:
def formatting_prompts_func(example):
    output_texts = []
    for conversation in example['conversation']:
        texts = []
        for line in conversation:
            text = f"### {'User' if line['role'] == 'speaker' else 'Assistant'}: {line['text']}{'</끝>' if line['role']!='speaker' else ''}"
            texts.append(text)
        output_texts.append("\n".join(texts))
    return output_texts

In [10]:
from pprint import pprint
pprint(formatting_prompts_func(dataset[10:11]))

KeyError: 'text'